# Data Analysis: RecipeDB
## 4.2 Network Analysis - Backbone Extraction
---

To extract the backbone structure of an undirected weighted network, Disparity Filter - a network reduction algorithm is used.


Disparity filter identifies the links that should be preserved in the network. 
<br>The Null hypothesis used is: normalized weights that correspond to the connections of a certain node of degree k are produced by a random assignment from a uniform distribution.
<br>By imposing a significance level $\alpha$, the links that carry weights that can be considered not compatible with a random distribution can be filtered out with a certain statistical significance.

The statistically relevant edges will be those whose weights satisfy the relation:<br>
$$\displaystyle\alpha_{ij} = 1 - (k - 1) \int_{0}^{p_{ij}}(1 - x)^{k-2}dx < \alpha$$
where,
<br>$k$: number of connections of the node to which the link under consideration is attached.
<br>$p_{ij}$: normalised weight.
<br>$\alpha_{ij}$: probability that the node's normalised weight $p_{ij}$ is compatible with the null hypothesis.

The multiscale backbone is then obtained by preserving all the links that satisfy the above criterion for at least one of the two nodes at the ends of the link while discounting the rest.

<br>

<em>Reference</em>: M. A. Serrano et al. (2009) Extracting the Multiscale backbone of complex weighted networks. PNAS, 106:16, pp. 6483-6488.
<br>The following code has been taken from: [aekpalakorn/python-backbone-network/blob/master/backbone.py](https://github.com/aekpalakorn/python-backbone-network/blob/master/backbone.py)

In [1]:
import networkx as nx
import numpy as np
from scipy import integrate
import matplotlib.pyplot as plt

In [2]:
'''
This module implements the disparity filter to compute a significance score of edge weights in networks
'''

def disparity_filter(G, weight='weight'):
    ''' Compute significance scores (alpha) for weighted edges in G as defined in Serrano et al. 2009
        Args
            G: Weighted NetworkX graph
        Returns
            Weighted graph with a significance score (alpha) assigned to each edge
        References
            M. A. Serrano et al. (2009) Extracting the Multiscale backbone of complex weighted networks. PNAS, 106:16, pp. 6483-6488.
    '''
    
    if nx.is_directed(G): #directed case    
        N = nx.DiGraph()
        for u in G:
            
            k_out = G.out_degree(u)
            k_in = G.in_degree(u)
            
            if k_out > 1:
                sum_w_out = sum(np.absolute(G[u][v][weight]) for v in G.successors(u))
                for v in G.successors(u):
                    w = G[u][v][weight]
                    p_ij_out = float(np.absolute(w))/sum_w_out
                    alpha_ij_out = 1 - (k_out-1) * integrate.quad(lambda x: (1-x)**(k_out-2), 0, p_ij_out)[0]
                    N.add_edge(u, v, weight = w, alpha_out=float('%.4f' % alpha_ij_out))
                    
            elif k_out == 1 and G.in_degree(G.successors(u)[0]) == 1:
                #we need to keep the connection as it is the only way to maintain the connectivity of the network
                v = G.successors(u)[0]
                w = G[u][v][weight]
                N.add_edge(u, v, weight = w, alpha_out=0., alpha_in=0.)
                #there is no need to do the same for the k_in, since the link is built already from the tail
            
            if k_in > 1:
                sum_w_in = sum(np.absolute(G[v][u][weight]) for v in G.predecessors(u))
                for v in G.predecessors(u):
                    w = G[v][u][weight]
                    p_ij_in = float(np.absolute(w))/sum_w_in
                    alpha_ij_in = 1 - (k_in-1) * integrate.quad(lambda x: (1-x)**(k_in-2), 0, p_ij_in)[0]
                    N.add_edge(v, u, weight = w, alpha_in=float('%.4f' % alpha_ij_in))
        return N
    
    else: #undirected case
        B = nx.Graph()
        for u in G:
            k = len(G[u])
            if k > 1:
                sum_w = sum(np.absolute(G[u][v][weight]) for v in G[u])
                for v in G[u]:
                    w = G[u][v][weight]
                    p_ij = float(np.absolute(w))/sum_w
                    alpha_ij = 1 - (k-1) * integrate.quad(lambda x: (1-x)**(k-2), 0, p_ij)[0]
                    B.add_edge(u, v, weight = w, alpha=float('%.4f' % alpha_ij))
        return B

def disparity_filter_alpha_cut(G,weight='weight',alpha_t=0.4, cut_mode='or'):
    ''' Performs a cut of the graph previously filtered through the disparity_filter function.
        
        Args
        ----
        G: Weighted NetworkX graph
        
        weight: string (default='weight')
            Key for edge data used as the edge weight w_ij.
            
        alpha_t: double (default='0.4')
            The threshold for the alpha parameter that is used to select the surviving edges.
            It has to be a number between 0 and 1.
            
        cut_mode: string (default='or')
            Possible strings: 'or', 'and'.
            It works only for directed graphs. It represents the logic operation to filter out edges
            that do not pass the threshold value, combining the alpha_in and alpha_out attributes
            resulting from the disparity_filter function.
            
            
        Returns
        -------
        B: Weighted NetworkX graph
            The resulting graph contains only edges that survived from the filtering with the alpha_t threshold
    
        References
        ---------
        .. M. A. Serrano et al. (2009) Extracting the Multiscale backbone of complex weighted networks. PNAS, 106:16, pp. 6483-6488.
    '''    
    
    if nx.is_directed(G):#Directed case:   
        B = nx.DiGraph()
        for u, v, w in G.edges(data=True):
            try:
                alpha_in =  w['alpha_in']
            except KeyError: #there is no alpha_in, so we assign 1. It will never pass the cut
                alpha_in = 1
            try:
                alpha_out =  w['alpha_out']
            except KeyError: #there is no alpha_out, so we assign 1. It will never pass the cut
                alpha_out = 1  
            
            if cut_mode == 'or':
                if alpha_in<alpha_t or alpha_out<alpha_t:
                    B.add_edge(u,v, weight=w[weight])
            elif cut_mode == 'and':
                if alpha_in<alpha_t and alpha_out<alpha_t:
                    B.add_edge(u,v, weight=w[weight])
        return B

    else:
        B = nx.Graph()#Undirected case:   
        for u, v, w in G.edges(data=True):
            
            try:
                alpha = w['alpha']
            except KeyError: #there is no alpha, so we assign 1. It will never pass the cut
                alpha = 1
                
            if alpha<alpha_t:
                B.add_edge(u,v, weight=w[weight])
        return B          

We will now read the file ingredient_weights.csv that stores the weighted graph of ingredients having common recipes. 

In [3]:
if __name__ == '__main__':

    #read the file
    G = nx.read_edgelist('output_files/ingredient_weights.csv', delimiter=',' , encoding='latin1',create_using=nx.Graph(),nodetype=str,data=(('weight',int),))
    alpha = 0.05
    
    #use the disparity filter algorithm
    G = disparity_filter(G)
    G2 = nx.Graph([(u, v, d) for u, v, d in G.edges(data=True) if d['alpha'] < alpha])
    print('alpha =', alpha)
    print('The extracted backbone structure contains: \n{0:.2f}% of the original nodes \n{1:.2f}% of the original edges'.format((G2.number_of_nodes()/G.number_of_nodes())*100, (G2.number_of_edges()/G.number_of_edges())*100))

alpha = 0.05
The extracted backbone structure contains: 
22.44% of the original nodes 
5.71% of the original edges


If we take $\alpha = 0.05$, the extracted backbone structure contains: 
- 21.52% of the original nodes 
- 5.74% of the original edges

Thus, we can significantly reduce the components in our network while simultaneously preserving the backbone structure.